# Geração de Pares de Perguntas e SQL

## Pipelinde de Aumento de Dados

**Fonte:**
Li, H., Zhang, J., Liu, H., Fan, J., Zhang, X., Zhu, J., Wei, R., Pan, H., Li, C., & Chen, H. (2024). CodeS: Towards building open-source language models for text-to-SQL. Proceedings of the ACM on Management of Data, 2(3), Article 127. https://doi.org/10.1145/3654930

A abordagem parte de um conjunto inicial de pares (pergunta em linguagem natural, consulta SQL correspondente) e busca expandi-lo para gerar uma base mais ampla e representativa. Essa expansão foi pensada para especializar modelos de Text-to-SQL em domínios específicos, como o nosso caso com o ecossistema do DataSUS.


### Etapas do Pipeline

<img src="assets/codesPipeline.png" width="500">

#### **Conjunto Inicial**
Reúne-se um pequeno conjunto de pares (pergunta, SQL) que servirá como referência para expansão.

#### **Geração de Novas Perguntas**
Um modelo gera variações de perguntas baseadas nas originais, preservando a intenção, mas aumentando a diversidade.

#### **Geração das Consultas SQL Sintetizadas**
O mesmo modelo produz a consulta SQL correspondente para cada nova pergunta gerada.

#### **Conjunto Ampliado**
O resultado é um conjunto maior de pares realistas, adequado para treinar ou especializar modelos de Text-to-SQL para o domínio do DataSUS.

## Como Estamos Fazendo

<img src="assets/ourPipeline.png" width="500">

A diferença é que utilizamos LLMs para gerar um conjunto de perguntas em linguagem natural que fizessem sentido com os datasets do DataSUS. Com isso, fizemos um processo de curadoria, onde retirávamos consultas muito similares, perguntas que não faziam sentido ou que não estavam de acordo com o schema do banco de dados.

In [1]:
from sample import show_queries, selected_queries

show_queries(selected_queries)

╭────────────────────────────────────────────────── Consulta 1 ───────────────────────────────────────────────────╮
│ NL Query: Liste todos os estabelecimentos de saúde em São Paulo que possuem leitos de UTI Adulto disponíveis    │
│ para o SUS na última competência disponível.                                                                    │
│                                                                                                                 │
│ Identificar hospitais de São Paulo com disponibilidade atual de leitos de UTI adulto via SUS.                   │
│                                                                                                                 │
│   1                                                                                                             │
│   2 SELECT nome_hospital, cnes                                                                                  │
│   3 FROM leitos                                                                                                 │
│   4 WHERE uf_hospital = 'SP'                                                                                    │
│   5   AND UTI_sus_adulto > 0                                                                                    │
│   6   AND data_competencia_info = (SELECT MAX(data_competencia_info) FROM leitos);                              │
│   7                                                                                                             │
│                                                                                                                 │
│  Complexidade:  simple                                                                                          │
│  Tipo:          filter                                                                                          │
│  Tabelas:       leitos                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Consulta 2 ───────────────────────────────────────────────────╮
│ NL Query: Liste os municípios na Bahia onde a média de leitos de UTI Neonatal SUS por estabelecimento é         │
│ superior à média estadual de leitos de UTI Neonatal SUS por estabelecimento, considerando os dados do último    │
│ ano completo.                                                                                                   │
│                                                                                                                 │
│ Localizar municípios da Bahia que possuem oferta de UTI neonatal acima da média estadual.                       │
│                                                                                                                 │
│    1                                                                                                            │
│    2 SELECT                                                                                                     │
│    3   municipio_hospital,                                                                                      │
│    4   AVG(UTI_sus_neonatal) AS media_municipal                                                                 │
│    5 FROM leitos                                                                                                │
│    6 WHERE uf_hospital = 'BA' AND SUBSTRING(data_competencia_info, 1, 4)::integer = (SELECT MAX(SUBSTRING(data_ │
│    7 GROUP BY municipio_hospital                                                                                │
│    8 HAVING AVG(UTI_sus_neonatal) > (                                                                           │
│    9   SELECT AVG(UTI_sus_neonatal)                                                                             │
│   10   FROM leitos                                                                                              │
│   11   WHERE uf_hospital = 'BA'                                                                                 │
│   12     AND SUBSTRING(data_competencia_info, 1, 4)::integer = (                                                │
│   13       SELECT MAX(SUBSTRING(data_competencia_info, 1, 4)::integer)                                          │
│   14       FROM leitos                                                                                          │
│   15       WHERE uf_hospital = 'BA'));                                                                          │
│   16                                                                                                            │
│                                                                                                                 │
│  Complexidade:  complex                                                                                         │
│  Tipo:          subquery                                                                                        │
│  Tabelas:       leitos                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Consulta 3 ───────────────────────────────────────────────────╮
│ NL Query: Quais são os produtos (descrição CATMAT) que foram comprados por instituições localizadas em          │
│ municípios que possuíam mais de 200 leitos existentes no ano de 2023, mas que tiveram seu preço unitário médio  │
│ de compra superior à média nacional para o mesmo produto?                                                       │
│                                                                                                                 │
│ Detectar produtos comprados por municípios grandes em 2023 cujo preço médio ficou acima da média nacional.      │
│                                                                                                                 │
│    1                                                                                                            │
│    2 WITH MediaNacional AS (                                                                                    │
│    3   SELECT descricao_catmat, AVG(preco_unitario) AS media_nacional                                           │
│    4   FROM bps                                                                                                 │
│    5   WHERE ano_compra = 2023                                                                                  │
│    6   GROUP BY descricao_catmat                                                                                │
│    7 ),                                                                                                         │
│    8 MunicipiosComLeitos AS (                                                                                   │
│    9   SELECT municipio_hospital, uf_hospital                                                                   │
│   10   FROM leitos                                                                                              │
│   11   WHERE SUBSTRING(data_competencia_info, 1, 4) = '2023'                                                    │
│   12   GROUP BY municipio_hospital, uf_hospital                                                                 │
│   13   HAVING SUM(leitos_geral) > 200                                                                           │
│   14 )                                                                                                          │
│   15 SELECT DISTINCT b.descricao_catmat                                                                         │
│   16 FROM bps b                                                                                                 │
│   17 JOIN MediaNacional mn                                                                                      │
│   18   ON b.descricao_catmat = mn.descricao_catmat                                                              │
│   19 WHERE b.ano_compra = 2023                                                                                  │
│   20   AND (b.municipio, b.uf_instituicao) IN (                                                                 │
│   21     SELECT municipio_hospital, uf_hospital FROM MunicipiosComLeitos                                        │
│   22   )                                                                                                        │
│   23   AND b.preco_unitario > mn.media_nacional;                                                                │
│   24                                                                                                            │
│                                                                                                                 │
│  Complexidade:  complex                                                                                         │
│  Tipo:          join                                                                                            │
│  Tabelas:       bps, leitos                           

╭────────────────────────────────────────────────── Consulta 4 ───────────────────────────────────────────────────╮
│ NL Query: Para cada CNES, qual o produto com a maior disparidade entre o preço unitário médio de compra e o     │
│ preço unitário mínimo de compra nos últimos 12 meses?                                                           │
│                                                                                                                 │
│ Encontrar para cada CNES o item com maior variação entre preço médio e mínimo no último ano.                    │
│                                                                                                                 │
│    1                                                                                                            │
│    2 WITH DisparidadePreco AS (                                                                                 │
│    3   SELECT                                                                                                   │
│    4     bn.codigo_cnes,                                                                                        │
│    5     bp.descricao_catmat,                                                                                   │
│    6     AVG(bp.preco_unitario) - MIN(bp.preco_unitario) AS disparidade,                                        │
│    7     ROW_NUMBER() OVER (                                                                                    │
│    8       PARTITION BY bn.codigo_cnes                                                                          │
│    9       ORDER BY AVG(bp.preco_unitario) - MIN(bp.preco_unitario) DESC                                        │
│   10     ) AS rn                                                                                                │
│   11   FROM bps bp                                                                                              │
│   12   JOIN bnafar bn                                                                                           │
│   13     ON bp.codigo_br = bn.co_catmat                                                                         │
│   14   WHERE bp.data_compra >= CURRENT_DATE - INTERVAL '12' MONTH                                               │
│   15   GROUP BY bn.codigo_cnes, bp.descricao_catmat                                                             │
│   16 )                                                                                                          │
│   17 SELECT                                                                                                     │
│   18   codigo_cnes,                                                                                             │
│   19   descricao_catmat,                                                                                        │
│   20   disparidade                                                                                              │
│   21 FROM DisparidadePreco                                                                                      │
│   22 WHERE rn = 1;                                                                                              │
│   23                                                                                                            │
│                                                                                                                 │
│  Complexidade:  complex                                                                                         │
│  Tipo:          join                                                                                            │
│  Tabelas:       bps, bnafar                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Consulta 5 ───────────────────────────────────────────────────╮
│ NL Query: Quais os 10 medicamentos com o maior preço unitário médio nas compras realizadas no ano de 2023, e    │
│ qual o nome do fornecedor mais comum para cada um desses itens?                                                 │
│                                                                                                                 │
│ Identificar os medicamentos mais caros de 2023 e seus fornecedores mais frequentes.                             │
│                                                                                                                 │
│    1                                                                                                            │
│    2 WITH PrecoMedio AS (                                                                                       │
│    3   SELECT                                                                                                   │
│    4     descricao_catmat,                                                                                      │
│    5     AVG(preco_unitario) AS preco_medio                                                                     │
│    6   FROM bps                                                                                                 │
│    7   WHERE ano_compra = 2023                                                                                  │
│    8   GROUP BY descricao_catmat                                                                                │
│    9   ORDER BY preco_medio DESC                                                                                │
│   10   LIMIT 10                                                                                                 │
│   11 ),                                                                                                         │
│   12 FornecedorComum AS (                                                                                       │
│   13   SELECT                                                                                                   │
│   14     descricao_catmat,                                                                                      │
│   15     fornecedor,                                                                                            │
│   16     ROW_NUMBER() OVER (                                                                                    │
│   17       PARTITION BY descricao_catmat                                                                        │
│   18       ORDER BY COUNT(*) DESC                                                                               │
│   19     ) AS rn                                                                                                │
│   20   FROM bps                                                                                                 │
│   21   WHERE ano_compra = 2023                                                                                  │
│   22     AND descricao_catmat IN (SELECT descricao_catmat FROM PrecoMedio)                                      │
│   23   GROUP BY descricao_catmat, fornecedor                                                                    │
│   24 )                                                                                                          │
│   25 SELECT                                                                                                     │
│   26   pm.descricao_catmat,                                                                                     │
│   27   pm.preco_medio,                                                                                          │
│   28   fc.fornecedor AS fornecedor_mais_comum                                                                   │
│   29 FROM PrecoMedio pm                               

### Aumento de Dados

Utilizamos um LLM para gerar 5 paráfrases para cada pergunta original, dando flexibilidade para mudar a estrutura da frase, mas mantendo o significado. Também deixamos livres para o modelo modificar valores específicos (como anos/números) para aumentar a diversidade das perguntas, sem contar que configuramos uma temperatura mais alta para incentivar uma geração mais variada.

Exemplos de gerações:

In [2]:
from sample import show_generated_queries, generated_queries

show_generated_queries(generated_queries)

╭─────────────────────────────────────────────── Consulta Gerada 1 ───────────────────────────────────────────────╮
│ Original NL Query: Liste todos os estabelecimentos de saúde em São Paulo que possuem leitos de UTI Adulto       │
│ disponíveis para o SUS na última competência disponível.                                                        │
│                                                                                                                 │
│ NL Query Reformulada: Mostre todos os hospitais em São Paulo que têm leitos de UTI Adulto disponíveis para o    │
│ SUS na última competência registrada.                                                                           │
│                                                                                                                 │
│   1 SELECT nome_hospital,                                                                                       │
│   2        cnes                                                                                                 │
│   3 FROM leitos                                                                                                 │
│   4 WHERE uf_hospital = 'SP'                                                                                    │
│   5   AND UTI_sus_adulto > 0                                                                                    │
│   6   AND data_competencia_info =                                                                               │
│   7     (SELECT MAX(data_competencia_info)                                                                      │
│   8      FROM leitos);                                                                                          │
│                                                                                                                 │
│  Complexidade:  simple                                                                                          │
│  Tipo:          filter                                                                                          │
│  Tabelas:       leitos                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Consulta Gerada 2 ───────────────────────────────────────────────╮
│ Original NL Query: Liste todos os estabelecimentos de saúde em São Paulo que possuem leitos de UTI Adulto       │
│ disponíveis para o SUS na última competência disponível.                                                        │
│                                                                                                                 │
│ NL Query Reformulada: Quero ver a lista de estabelecimentos de saúde em São Paulo com leitos de UTI Adulto para │
│ o SUS na mais recente competência.                                                                              │
│                                                                                                                 │
│   1 SELECT nome_estabelecimento,                                                                                │
│   2        cnes                                                                                                 │
│   3 FROM leitos                                                                                                 │
│   4 WHERE uf_hospital = 'SP'                                                                                    │
│   5   AND UTI_sus_adulto > 0                                                                                    │
│   6   AND data_competencia_info =                                                                               │
│   7     (SELECT MAX(data_competencia_info)                                                                      │
│   8      FROM leitos);                                                                                          │
│                                                                                                                 │
│  Complexidade:  simple                                                                                          │
│  Tipo:          filter                                                                                          │
│  Tabelas:       leitos                                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Consulta Gerada 3 ───────────────────────────────────────────────╮
│ Original NL Query: Crie um ranking dos 5 fabricantes que venderam itens para o maior número de instituições     │
│ compradoras diferentes no último ano, e qual o valor total de vendas desses fabricantes.                        │
│                                                                                                                 │
│ NL Query Reformulada: Liste os 5 principais fabricantes que fornecem produtos para o maior número de            │
│ instituições compradoras distintas no último ano e informe o total em vendas desses fabricantes.                │
│                                                                                                                 │
│   1 SELECT fabricante,                                                                                          │
│   2        COUNT(DISTINCT cnpj_instituicao) AS num_instituicoes_compradoras,                                    │
│   3        SUM(preco_total) AS valor_total_vendas                                                               │
│   4 FROM bps                                                                                                    │
│   5 WHERE ano_compra = EXTRACT(YEAR                                                                             │
│   6                            FROM CURRENT_DATE) - 1                                                           │
│   7 GROUP BY fabricante                                                                                         │
│   8 ORDER BY valor_total_vendas DESC                                                                            │
│   9 LIMIT 5;                                                                                                    │
│                                                                                                                 │
│  Complexidade:  complex                                                                                         │
│  Tipo:          aggregation                                                                                     │
│  Tabelas:       bps                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Consulta Gerada 4 ───────────────────────────────────────────────╮
│ Original NL Query: Crie um ranking dos 5 fabricantes que venderam itens para o maior número de instituições     │
│ compradoras diferentes no último ano, e qual o valor total de vendas desses fabricantes.                        │
│                                                                                                                 │
│ NL Query Reformulada: Elabore uma lista dos 3 fabricantes com o maior número de instituições compradoras        │
│ diferentes que realizaram compras no último ano e qual foi o total de vendas destes.                            │
│                                                                                                                 │
│   1 SELECT fabricante,                                                                                          │
│   2        COUNT(DISTINCT cnpj_instituicao) AS num_instituicoes_compradoras,                                    │
│   3        SUM(preco_total) AS valor_total_vendas                                                               │
│   4 FROM bps                                                                                                    │
│   5 WHERE ano_compra = EXTRACT(YEAR                                                                             │
│   6                            FROM CURRENT_DATE) - 1                                                           │
│   7 GROUP BY fabricante                                                                                         │
│   8 ORDER BY num_instituicoes_compradoras DESC                                                                  │
│   9 LIMIT 3;                                                                                                    │
│                                                                                                                 │
│  Complexidade:  complex                                                                                         │
│  Tipo:          aggregation                                                                                     │
│  Tabelas:       bps                                                                                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Situação Atual

Tínhamos um conjunto inicial de 20 pares. Com esse pipelines, conseguimos aumentar para 116.

### Avaliação da Qualidade das Paráfrases
Para fazer a avaliação da qualidade das paráfrases geradas, utilizamos três métricas distintas: Cross-Encoder, SBERT e BLEU. A seguir, apresentamos uma análise detalhada dos resultados obtidos com cada uma dessas métricas.

In [3]:
from ParaphraseEvaluator import ParaphraseEvaluator
from generated_queries import generated_queries

pairs = [(x["original_nl_query"], x["nl_query"]) for x in generated_queries]

evaluator = ParaphraseEvaluator()
evaluator.evaluate_and_print(pairs)

/Users/jaidezardin/Library/Caches/pypoetry/virtualenvs/project-embrapii-0MF4zvhF-py3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


    Cross-Encoder: Análise por Query    
                Original                
┏━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓
┃ ID ┃ N Paráfrases ┃  Média ┃ Mediana ┃
┡━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩
│ 1  │      5       │ 0.7978 │  0.7860 │
├────┼──────────────┼────────┼─────────┤
│ 2  │      5       │ 0.8360 │  0.8749 │
├────┼──────────────┼────────┼─────────┤
│ 3  │      5       │ 0.8848 │  0.8752 │
├────┼──────────────┼────────┼─────────┤
│ 4  │      5       │ 0.8719 │  0.8742 │
├────┼──────────────┼────────┼─────────┤
│ 5  │      1       │ 0.6987 │  0.6987 │
├────┼──────────────┼────────┼─────────┤
│ 6  │      5       │ 0.7732 │  0.7825 │
├────┼──────────────┼────────┼─────────┤
│ 7  │      5       │ 0.7612 │  0.7863 │
├────┼──────────────┼────────┼─────────┤
│ 8  │      5       │ 0.8219 │  0.8294 │
├────┼──────────────┼────────┼─────────┤
│ 9  │      5       │ 0.8480 │  0.8841 │
├────┼──────────────┼────────┼─────────┤
│ 10 │      5       │ 0.7007 │  0.7754 │
├────┼──────────────┼────────┼─────────┤
│ 11 │      5       │ 0.7674 │  0.7721 │
├────┼──────────────┼────────┼─────────┤
│ 12 │      5       │ 0.6862 │  0.6734 │
├────┼──────────────┼────────┼─────────┤
│ 13 │      5       │ 0.8405 │  0.8636 │
├────┼──────────────┼────────┼─────────┤
│ 14 │      5       │ 0.7928 │  0.7959 │
├────┼──────────────┼────────┼─────────┤
│ 15 │      5       │ 0.7432 │  0.7436 │
├────┼──────────────┼────────┼─────────┤
│ 16 │      5       │ 0.8084 │  0.8247 │
├────┼──────────────┼────────┼─────────┤
│ 17 │      4       │ 0.7436 │  0.7440 │
├────┼──────────────┼────────┼─────────┤
│ 18 │      5       │ 0.8097 │  0.8068 │
├────┼──────────────┼────────┼─────────┤
│ 19 │      5       │ 0.8533 │  0.8473 │
├────┼──────────────┼────────┼─────────┤
│ 20 │      5       │ 0.8966 │  0.8885 │
└────┴──────────────┴────────┴─────────┘

  Cross-Encoder: Estatísticas Gerais   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Métrica                    ┃  Valor ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Total de queries originais │     20 │
│ Total de paráfrases        │     95 │
│ Média geral dos scores     │ 0.8015 │
│ Mediana geral dos scores   │ 0.8059 │
│ Desvio padrão geral        │ 0.0877 │
└────────────────────────────┴────────┘

    Sentence-BERT: Análise por Query    
                Original                
┏━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓
┃ ID ┃ N Paráfrases ┃  Média ┃ Mediana ┃
┡━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩
│ 1  │      5       │ 0.9053 │  0.9042 │
├────┼──────────────┼────────┼─────────┤
│ 2  │      5       │ 0.9199 │  0.9266 │
├────┼──────────────┼────────┼─────────┤
│ 3  │      5       │ 0.9373 │  0.9410 │
├────┼──────────────┼────────┼─────────┤
│ 4  │      5       │ 0.8897 │  0.8839 │
├────┼──────────────┼────────┼─────────┤
│ 5  │      1       │ 0.7667 │  0.7667 │
├────┼──────────────┼────────┼─────────┤
│ 6  │      5       │ 0.9079 │  0.8966 │
├────┼──────────────┼────────┼─────────┤
│ 7  │      5       │ 0.7764 │  0.7620 │
├────┼──────────────┼────────┼─────────┤
│ 8  │      5       │ 0.9193 │  0.9109 │
├────┼──────────────┼────────┼─────────┤
│ 9  │      5       │ 0.8949 │  0.9151 │
├────┼──────────────┼────────┼─────────┤
│ 10 │      5       │ 0.7900 │  0.7985 │
├────┼──────────────┼────────┼─────────┤
│ 11 │      5       │ 0.8907 │  0.9131 │
├────┼──────────────┼────────┼─────────┤
│ 12 │      5       │ 0.9250 │  0.9263 │
├────┼──────────────┼────────┼─────────┤
│ 13 │      5       │ 0.8323 │  0.9347 │
├────┼──────────────┼────────┼─────────┤
│ 14 │      5       │ 0.8705 │  0.8577 │
├────┼──────────────┼────────┼─────────┤
│ 15 │      5       │ 0.8985 │  0.8997 │
├────┼──────────────┼────────┼─────────┤
│ 16 │      5       │ 0.9124 │  0.9220 │
├────┼──────────────┼────────┼─────────┤
│ 17 │      4       │ 0.8922 │  0.9160 │
├────┼──────────────┼────────┼─────────┤
│ 18 │      5       │ 0.9562 │  0.9561 │
├────┼──────────────┼────────┼─────────┤
│ 19 │      5       │ 0.8750 │  0.8703 │
├────┼──────────────┼────────┼─────────┤
│ 20 │      5       │ 0.9416 │  0.9436 │
└────┴──────────────┴────────┴─────────┘

  Sentence-BERT: Estatísticas Gerais   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Métrica                    ┃  Valor ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Total de queries originais │     20 │
│ Total de paráfrases        │     95 │
│ Média geral dos scores     │ 0.8900 │
│ Mediana geral dos scores   │ 0.9109 │
│ Desvio padrão geral        │ 0.0671 │
└────────────────────────────┴────────┘

 BLEU Score: Análise por Query Original 
┏━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓
┃ ID ┃ N Paráfrases ┃  Média ┃ Mediana ┃
┡━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩
│ 1  │      5       │ 0.4856 │  0.5283 │
├────┼──────────────┼────────┼─────────┤
│ 2  │      5       │ 0.3086 │  0.2869 │
├────┼──────────────┼────────┼─────────┤
│ 3  │      5       │ 0.4265 │  0.4152 │
├────┼──────────────┼────────┼─────────┤
│ 4  │      5       │ 0.2103 │  0.1818 │
├────┼──────────────┼────────┼─────────┤
│ 5  │      1       │ 0.1127 │  0.1127 │
├────┼──────────────┼────────┼─────────┤
│ 6  │      5       │ 0.3671 │  0.3382 │
├────┼──────────────┼────────┼─────────┤
│ 7  │      5       │ 0.1656 │  0.1654 │
├────┼──────────────┼────────┼─────────┤
│ 8  │      5       │ 0.2810 │  0.3125 │
├────┼──────────────┼────────┼─────────┤
│ 9  │      5       │ 0.4232 │  0.4456 │
├────┼──────────────┼────────┼─────────┤
│ 10 │      5       │ 0.2882 │  0.2955 │
├────┼──────────────┼────────┼─────────┤
│ 11 │      5       │ 0.3291 │  0.3218 │
├────┼──────────────┼────────┼─────────┤
│ 12 │      5       │ 0.4978 │  0.4877 │
├────┼──────────────┼────────┼─────────┤
│ 13 │      5       │ 0.4330 │  0.3949 │
├────┼──────────────┼────────┼─────────┤
│ 14 │      5       │ 0.1623 │  0.1184 │
├────┼──────────────┼────────┼─────────┤
│ 15 │      5       │ 0.2356 │  0.2230 │
├────┼──────────────┼────────┼─────────┤
│ 16 │      5       │ 0.2373 │  0.2331 │
├────┼──────────────┼────────┼─────────┤
│ 17 │      4       │ 0.3785 │  0.3810 │
├────┼──────────────┼────────┼─────────┤
│ 18 │      5       │ 0.3192 │  0.3198 │
├────┼──────────────┼────────┼─────────┤
│ 19 │      5       │ 0.2755 │  0.2600 │
├────┼──────────────┼────────┼─────────┤
│ 20 │      5       │ 0.4698 │  0.5161 │
└────┴──────────────┴────────┴─────────┘

    BLEU Score: Estatísticas Gerais    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Métrica                    ┃  Valor ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Total de queries originais │     20 │
│ Total de paráfrases        │     95 │
│ Média geral dos scores     │ 0.3285 │
│ Mediana geral dos scores   │ 0.3225 │
│ Desvio padrão geral        │ 0.1386 │
└────────────────────────────┴────────┘

            Comparação das Três Métricas por Query Original             
┏━━━━┳━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓
┃    ┃   ┃ Cross-Enc ┃ Cross-Enc ┃  SBERT ┃   SBERT ┃   BLEU ┃    BLEU ┃
┃ ID ┃ N ┃     Média ┃   Mediana ┃  Média ┃ Mediana ┃  Média ┃ Mediana ┃
┡━━━━╇━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩
│ 1  │ 5 │    0.7978 │    0.7860 │ 0.9053 │  0.9042 │ 0.4856 │  0.5283 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 2  │ 5 │    0.8360 │    0.8749 │ 0.9199 │  0.9266 │ 0.3086 │  0.2869 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 3  │ 5 │    0.8848 │    0.8752 │ 0.9373 │  0.9410 │ 0.4265 │  0.4152 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 4  │ 5 │    0.8719 │    0.8742 │ 0.8897 │  0.8839 │ 0.2103 │  0.1818 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 5  │ 1 │    0.6987 │    0.6987 │ 0.7667 │  0.7667 │ 0.1127 │  0.1127 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 6  │ 5 │    0.7732 │    0.7825 │ 0.9079 │  0.8966 │ 0.3671 │  0.3382 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 7  │ 5 │    0.7612 │    0.7863 │ 0.7764 │  0.7620 │ 0.1656 │  0.1654 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 8  │ 5 │    0.8219 │    0.8294 │ 0.9193 │  0.9109 │ 0.2810 │  0.3125 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 9  │ 5 │    0.8480 │    0.8841 │ 0.8949 │  0.9151 │ 0.4232 │  0.4456 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 10 │ 5 │    0.7007 │    0.7754 │ 0.7900 │  0.7985 │ 0.2882 │  0.2955 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 11 │ 5 │    0.7674 │    0.7721 │ 0.8907 │  0.9131 │ 0.3291 │  0.3218 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 12 │ 5 │    0.6862 │    0.6734 │ 0.9250 │  0.9263 │ 0.4978 │  0.4877 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 13 │ 5 │    0.8405 │    0.8636 │ 0.8323 │  0.9347 │ 0.4330 │  0.3949 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 14 │ 5 │    0.7928 │    0.7959 │ 0.8705 │  0.8577 │ 0.1623 │  0.1184 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 15 │ 5 │    0.7432 │    0.7436 │ 0.8985 │  0.8997 │ 0.2356 │  0.2230 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 16 │ 5 │    0.8084 │    0.8247 │ 0.9124 │  0.9220 │ 0.2373 │  0.2331 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 17 │ 4 │    0.7436 │    0.7440 │ 0.8922 │  0.9160 │ 0.3785 │  0.3810 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 18 │ 5 │    0.8097 │    0.8068 │ 0.9562 │  0.9561 │ 0.3192 │  0.3198 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 19 │ 5 │    0.8533 │    0.8473 │ 0.8750 │  0.8703 │ 0.2755 │  0.2600 │
├────┼───┼───────────┼───────────┼────────┼─────────┼────────┼─────────┤
│ 20 │ 5 │    0.8966 │    0.8885 │ 0.9416 │  0.9436 │ 0.4698 │  0.5161 │
└────┴───┴───────────┴───────────┴────────┴─────────┴────────┴─────────┘

## Conclusão

Interpretamos que quanto menor a pontuação do BLEU, maior a diversidade da paráfrase em relação à pergunta original. Já as pontuações do Cross-Encoder e SBERT indicam o quão semelhante semanticamente a paráfrase está em relação à pergunta original, ou seja, quanto maior a pontuação, mais similar semanticamente são as frases.

Nosso objetivo era gerar paráfrases que fossem semanticamente semelhantes às perguntas originais, mas que apresentassem uma diversidade estrutural significativa. Com base nas pontuações obtidas, podemos concluir que o pipeline de aumento de dados foi eficaz em atingir esse objetivo. As paráfrases geradas apresentaram uma boa similaridade semântica com as perguntas originais, conforme indicado pelas pontuações do Cross-Encoder e SBERT, ao mesmo tempo em que mantiveram uma diversidade estrutural, evidenciada pela pontuação do BLEU. Isso sugere que o método utilizado para gerar as paráfrases foi bem-sucedido em equilibrar a fidelidade semântica com a diversidade linguística, resultando em um conjunto de dados enriquecido e variado.
g